# Core concepts

In this notebook, I will introduce and define the core concepts relevant for Orcabridge and Orcapod. These concepts will be used throughout the Orca ecosystem and in the other tutorial notebooks, with many having directly corresponding Python classes/objects.

Below I define few critical concepts for Orcapod.

* `Data` -- In Orcapod, the smallest unit of `data` is a single `file`. Unlike many other computation pipeline system, Orcapod pipeline in principle does **not** operate on `data` that's not a file. In other words, `Oracpod` pipeline will **not** pass a data in memory from one node to another. Consequently, all operations and processing in Orcapod pipeline revolves around `file` (NOTE: this is a particularly strong/restrictive version of Oracpod pipeline. We may consider extending data to things like environment variable and command line arguments)
* `Pathset` -- a unit of data that can be passed into a pod. A `pathset` consists of a file, a directory, or a collection of one or more file and directories.
* `Packet` -- a single concrete instance of key-value pair, mapping packet key to a single `pathset`.
* `Stream` -- a series of one or more `packets` flowing from a `data producer` to a `data consumer`. In a directed acyclic graph represneing an Orcapod `pipeline`, a `stream` corresponds to a *directed* edge connecting from a data source into a `data consumer` (e.g., `pod`)
* `Data producer` and `data consumer` -- in the Orcapod data pipeline, data (in form of `packet` of data flowing inside a `stream`) flows from a `data producer` to a `data consumer`. Consequentially, a `data consumer` may in turn act as a `data producer` downstream
* `Data source` -- Root level `data producer` (that is, the data originates from this `data producer` and it is not a `data consumer` of any stream). Typically `data source` is tied to a data storage, although you could have *procedural* `data source` where data packets are produced programatically.
* `Tag` -- each `Packet` in a stream *may be* associated with a `tag` that helps to assign semantic identity to the particular `packet`. For example, a data `packet` for an experimental data may be associated with a `tag` of session ID. Note that while `tag` provides a convenient and often meaningful ways of identifying and referring to specific packet within a stream, it should **not** be considered to be the defining identity of the `packet`. Identity of the `packet` is strictly determined by the exact data content of the `packet`, and not by how you refer to it. Consequently, it may be that two packets with an identical content (and thus shared identity) are associated with distinct `tags` in a `stream`. Conversely, an identical `tag` may be associated with two distinct `packets` in a stream.  Typically, one would associate a unique `tag` for each packet in the stream.
* `Operation` -- A *node* in the directed acyclic graph representing an Orcapod `pipeline`, corresponding to a step of data processing/transformation/computation. An `Operation` receives can be classified into either a `mapper` or a `pod` based on their role in `data provenance`.
* `Mapper` -- A class of `operation` that does **not** result in creation/alteration of a new data -- that is, `operation` does **not** every create or modify a file *content*. More specifically, `Mapper` operation can not produce a path that was not already present in the input streams to the `mapper`. This feature ensures that a `mapper` is fundamentally not involved in the reproducibility of computation. Consequently, `mapper` information is not necessary for the maintenance of proper `data provenance` in a tree of computation. However, `mapper` plays critical role in the actual execution of a data pipeline, determining which data `packet` will be fed into operations in the pipeline directed acyclic graph (DAG). Note that as long as it doesn't modify the content of any file, a `mapper` may inspect the content of any file in a `packet` it receives and alter its behavior based on the content of the file. In other words, `mapper` may alter what data file(s) gets passed around without changing/creating any file based on a rule that depends on `tag`, `packet` key (`argument` name) and/or file content.
* `Pod` (e.g. FunctionPod) -- fundamental unit of computation in Orcapod. `Pod` is the only class of `operation` that may create a new file. Critically, when operating within an Orcapod `pipeline`, a `pod` will **not** receive the `tag` information. Rather, `pod` must strictly operate on a single `packet`. An ideal `pod` will have completely deterministic behavior that only depends on the `packet` identity (that is, packet keys and `pathset` contents).

## What is Orcabrdige?

`Orcabridge` provide prototypal implementation of the above-defined key concepts in `orcapod`, with particular focus given to `stream`, `packet`, `tag`, `operation` (`pod` and `mapper`). This package provides the reference implementation of both synchronous and asynchronous `streams` as a sequence of `packets` associated with a `tag`. 


### Key limitations
Being a very basic reference implementation with a goal of serving as a playground for conceptual and algorithmic development, there are a few notable limitations in `orcabridge`, lacking some important features that would be found in the full implementation of `orcapod`.

* Limited `pathset` -- Only a very rudimentary implementation of a `pathset` can be found, where a typical `packet` would be a simple key-value pair where the value is a single file path. 
* `FunctionPod` in place of `pod` -- A ideal `pod` will represent a completely reproducible unit of computation with strict dependence on the input `packet` to the `pod`. However, in `orcabridge` is equivalated to a Python function -- even more specifically, the name assigned to the Python function. In otherwords, two `FunctionPods` are considered identical if they share the same function name. Quite obviously this is a gross simplification that can be trivially violated. However, for the intended purpose of `orcabridge`, this rather simplistic implementation/definition of `pod` should suffice to test all features in the `pipeline`. Consequently, to ensure that the pipeline operates as expected, **do not use the same name for two distinct functions**. Doing so will break the fundamental assumption of `FunctionPod` and lead to completely erradic behavior. On the other hand, it is ok to have one or more name used to refer to an identical function.
* Different pipeline DAG defintion -- In `orcapod` the directed acyclic graph (DAG) for the `pipeline` should be defined using YAML file (or less frequently using API on `pipeline` struct in the Rust library). In `orcabridge` you will find that a `pipeline` DAG is defined dynamically through a series of application of `operation`. This is very much akin to how some DAG-based neural network library like TensorFlow defines a computation graph. While this works well for simple examples, it is rather difficult to track changes to the pipeline defined dynamically/programmatically using version control system. Since *how* you define the pipeline DAG is strictly speaking an orthogonal problem to the everything else that concerns the operation of the `pipeline`, no effort will be given to align the DAG definition in `orcabridge` and `orcapod`.
* Limited usage of a `stream` -- Currently `orcabrdige` only support single producer single consumer (SCSP) `stream`x, whereas in `orcapod`, `stream` should support single producer multiple consumer (SPMC) paradigm. While the same stream can be used in multiple downstream operations, each iteration of the stream actually results in recomputations of the entire chain of pipeline leading up to that stream. This inefficiency can be ameliorated by `CacheStream` operation after particularly computationally expensive segment of the pipeline. Using storage-backed `FunctionPod` will also help ameliorate the cost of recomputation by retrieving memoized computation result instead of recomputing.